## Problem 3
Explore and cluster the neighborhoods in Toronto.
Generate maps to visualize your neighborhoods and how they cluster together

In [67]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
from bs4 import BeautifulSoup
import re

Libraries imported.


## 1. Scraping And Exploring Dataset
Scraping the data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M using BeautifulSoup and converting to a DataFrame

In [68]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [69]:
data=requests.get(url).text

In [70]:
soup=BeautifulSoup(data)

In [71]:
table=soup.find_all('table')

In [72]:
df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [73]:
lis1=[]
for i  in range(0,20):
        for j in range(0,9):
                lis1.append(table[0].find_all('tr')[i].find_all('td')[j].text.strip())

In [74]:
lis2=[]
for i in lis1:
    if len(i)>15:
        lis2.append(i)

In [75]:
PostalCode=[i[0:3] for i in lis2]
PostalCode
Borough=[]
Neighborhood=[]
lis3=[i[3:] for i in lis2]
for i in range(0,len(lis3)):
    Borough.extend(re.findall('.+(?=\()',lis3[i]))
    Neighborhood.extend(re.findall('(?<=\().+(?=\))',lis3[i]))

In [76]:
df['PostalCode']=PostalCode
df['Borough']=Borough
df['Neighborhood']=Neighborhood
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


Dataframe obtained from scraping

In [77]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [78]:
df.shape

(102, 3)

In [79]:
geodata=pd.read_csv(r"C:\Users\CHAD\Desktop\work\Coursera\IBM\Geospatial_Coordinates.csv")

In [80]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [81]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [82]:
df=df.rename(columns={'PostalCode':'Postal Code'})

In [83]:
new_Data=df.merge(geodata)

In [84]:
new_Data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Data with geodata

In [85]:
toronto_data=new_Data[new_Data.Borough.str.contains('Toronto')]
toronto_data.reset_index(drop=True,inplace=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [86]:
#EXPLORE THE FIRST NEIGHBORHOOD

In [87]:
CLIENT_ID = 'G5XELWRC2EMZHYGGKF20VTKEWZE3GSGFBAQZNG4NUYHKVHQT' # your Foursquare ID
CLIENT_SECRET = 'GKBXNMTGJTWF04VMQVCNDYG5WR4UCRFTQTA2VXFLKF1JIQAX' # your Foursquare Secret
ACCESS_TOKEN = 'BOLQGFEKLU1E0CTOOSVPC5SSUTRQSWPXZHYSYMNB4V33DE5C' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100

In [88]:
toronto_data.loc[0,'Neighborhood']

'Regent Park / Harbourfront'

In [89]:
address='Toronto, CA'
geolocator=Nominatim(user_agent="ny_explorer")
location= geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [90]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(toronto_data["Latitude"],toronto_data["Longitude"],toronto_data["Borough"],toronto_data["Neighborhood"]):
    label='{}{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,color='blue',popup=label,fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_toronto)
map_toronto


In [91]:
CLIENT_ID = 'G5XELWRC2EMZHYGGKF20VTKEWZE3GSGFBAQZNG4NUYHKVHQT' # your Foursquare ID
CLIENT_SECRET = 'GKBXNMTGJTWF04VMQVCNDYG5WR4UCRFTQTA2VXFLKF1JIQAX' # your Foursquare Secret
ACCESS_TOKEN = 'BOLQGFEKLU1E0CTOOSVPC5SSUTRQSWPXZHYSYMNB4V33DE5C' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G5XELWRC2EMZHYGGKF20VTKEWZE3GSGFBAQZNG4NUYHKVHQT
CLIENT_SECRET:GKBXNMTGJTWF04VMQVCNDYG5WR4UCRFTQTA2VXFLKF1JIQAX


In [92]:
#Exploring first neighborhood in DataFrame

In [93]:
toronto_data.loc[0,'Neighborhood']

'Regent Park / Harbourfront'

In [94]:
#Getting longitude and latitude

In [95]:
neighborhood_longitude=toronto_data.loc[0,'Longitude']
neighborhood_latitude=toronto_data.loc[0,'Latitude']
neighborhood_name=toronto_data.loc[0,'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park / Harbourfront are 43.6542599, -79.3606359.


In [96]:
#TOP 100 VENUES IN REGENTPARK/HARBOURFRONT

In [97]:
limit=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&r={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=G5XELWRC2EMZHYGGKF20VTKEWZE3GSGFBAQZNG4NUYHKVHQT&client_secret=GKBXNMTGJTWF04VMQVCNDYG5WR4UCRFTQTA2VXFLKF1JIQAX&v=20180604&ll=43.6542599,-79.3606359&r=500&limit=100'

In [98]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
venues=requests.get(url).json()['response']['groups'][0]['items']

In [100]:
import json
from pandas import json_normalize # tranform JSON file into a pandas dataframe
nearby_venues=json_normalize(venues).head()

In [101]:
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.location.neighborhood,venue.venuePage.id
0,e-0-53b8466a498e83df908c3f21-0,0,"[{'summary': 'This spot is popular', 'type': '...",53b8466a498e83df908c3f21,Tandem Coffee,368 King St E,at Trinity St,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",122,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN
1,e-0-54ea41ad498e9a11e9e13308-1,0,"[{'summary': 'This spot is popular', 'type': '...",54ea41ad498e9a11e9e13308,Roselle Desserts,362 King St E,Trinity St,43.653447,-79.362017,"[{'label': 'display', 'lat': 43.65344672305267...",143,CA,Toronto,ON,Canada,"[362 King St E (Trinity St), Toronto ON M5A 1K...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],M5A 1K9,NaN,NaN
2,e-0-5612b1cc498e3dd742af0dc8-2,0,"[{'summary': 'This spot is popular', 'type': '...",5612b1cc498e3dd742af0dc8,Impact Kitchen,573 King St E,at St Lawrence St,43.656369,-79.356980,"[{'label': 'display', 'lat': 43.65636850543279...",376,CA,Toronto,ON,Canada,"[573 King St E (at St Lawrence St), Toronto ON...","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],M5A 4L3,NaN,NaN
3,e-0-4ad4c05ef964a520bff620e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c05ef964a520bff620e3,The Distillery Historic District,"btwn Front, Cherry, Gardiner & Parliament",NaN,43.650244,-79.359323,"[{'label': 'display', 'lat': 43.65024435658077...",459,CA,Toronto,ON,Canada,"[btwn Front, Cherry, Gardiner & Parliament, To...","[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",0,[],M5A 3C4,NaN,NaN
4,e-0-51ccc048498ec7792efc955e-4,0,"[{'summary': 'This spot is popular', 'type': '...",51ccc048498ec7792efc955e,Corktown Common,NaN,NaN,43.655618,-79.356211,"[{'label': 'display', 'lat': 43.65561779974973...",387,CA,NaN,NaN,Canada,[Canada],"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN


In [102]:
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']

In [103]:
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [104]:
nearby_venues['venue.categories'].head()

0    [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...
1    [{'id': '4bf58dd8d48988d16a941735', 'name': 'B...
2    [{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...
3    [{'id': '4deefb944765f83613cdba6e', 'name': 'H...
4    [{'id': '4bf58dd8d48988d163941735', 'name': 'P...
Name: venue.categories, dtype: object

In [105]:
nearby_venues['venue.categories']=nearby_venues['venue.categories'].apply(lambda x:x[0]['name'])

In [106]:
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Impact Kitchen,Restaurant,43.656369,-79.356980
3,The Distillery Historic District,Historic Site,43.650244,-79.359323
4,Corktown Common,Park,43.655618,-79.356211


In [107]:
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]

In [108]:
nearby_venues

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Impact Kitchen,Restaurant,43.656369,-79.356980
3,The Distillery Historic District,Historic Site,43.650244,-79.359323
4,Corktown Common,Park,43.655618,-79.356211


In [109]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Manhattan

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [110]:
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [111]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [112]:
len(toronto_data)

39

In [113]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
The Danforth  East
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosedale
Enclave of M5E
St. Jame

In [114]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [115]:
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                          60
Brockton / Parkdale Village / Exhibition Place                                                                       22
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport     17
Central Bay Street                                                                                                   61
Christie                                                                                                             16
Church and Wellesley                                                                                                 77
Commerce Court / Victoria Hotel                                                                                     100
Davisville                                                                                                           34
Davisville North           

In [116]:
#Number of unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 235 uniques categories.


## 3. Analyze Each Neighborhood

In [117]:
# one hot encoding
toronto_onehot=pd.get_dummies(toronto_venues['Venue Category'])

In [118]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[162]] + list(toronto_onehot.columns[:162])+list(toronto_onehot.columns[163:])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()


,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [119]:
toronto_onehot.shape

(1583, 235)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [120]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Let's print each neighborhood along with the top 5 most common venues


In [121]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1  Cocktail Bar  0.07
2        Bakery  0.05
3    Restaurant  0.03
4      Pharmacy  0.03


----Brockton / Parkdale Village / Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3       Burrito Place  0.05
4  Italian Restaurant  0.05


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1      Sandwich Place  0.07
2                Café  0.05
3  Italian Restaurant  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Coffee Shop  0.06
4     Baby St

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [123]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Jazz Club
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Nightclub,Climbing Gym,Restaurant,Burrito Place
2,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Pizza Place,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Poke Place
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Restaurant,Candy Store,Athletics & Sports,Coffee Shop


## 4. Cluster Neighborhoods Using K-Means

In [124]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [125]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Theater,Pub,Café,French Restaurant,Health Food Store,Historic Site
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Italian Restaurant,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Cosmetics Shop,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cocktail Bar,Restaurant,Gym,Lingerie Store,Farmers Market,Seafood Restaurant,American Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Jazz Club


In [126]:
toronto_merged["Cluster Labels"].value_counts()

0    33
1     3
4     1
3     1
2     1
Name: Cluster Labels, dtype: int64

Finally, let's visualize the resulting clusters

In [127]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### EXAMINE EACH CLUSTER

CLUSTER 0

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Theater,Pub,Café,French Restaurant,Health Food Store,Historic Site
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Italian Restaurant,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Cosmetics Shop,Ramen Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Cocktail Bar,Restaurant,Gym,Lingerie Store,Farmers Market,Seafood Restaurant,American Restaurant
3,East Toronto,0,Health Food Store,Trail,Pub,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Jazz Club
5,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Pizza Place,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Poke Place
6,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Restaurant,Candy Store,Athletics & Sports,Coffee Shop
7,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Restaurant,Gym,Clothing Store,Deli / Bodega,Hotel,Salad Place,Seafood Restaurant
8,West Toronto,0,Bakery,Pharmacy,Liquor Store,Supermarket,Bank,Bar,Middle Eastern Restaurant,Café,Music Venue,Recording Studio
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Sporting Goods Shop,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery


CLUSTER 1

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
29,Central Toronto,1,Park,Tennis Court,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
33,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


CLUSTER 2

In [130]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East YorkEast Toronto,2,Park,Convenience Store,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


CLUSTER 4

In [131]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


CLUSTER 4

In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Home Service,Garden,Fast Food Restaurant,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
